In [40]:
# Importing necessary libraries
import cv2
import mediapipe as mp
import numpy as np
import os
import io
import matplotlib.pyplot as plt
from PIL import Image
from src.mediapipe import GooMedia
from src.utils import image_resize
from src.visualize import draw_face_one_color

In [41]:
def calculate_EAR(eye_points):
    eye_points = np.array(eye_points)
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [42]:
def detect_pupil(eye_gray, eye_color, iris_diameter):
    avg_intensity = np.mean(eye_gray)
    threshold_value = avg_intensity * 0.8
    _, thresh = cv2.threshold(eye_gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.medianBlur(thresh, 5)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        M = cv2.moments(c)
        if M['m00'] != 0:
            cX, cY = int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])
            center = (cX, cY)
            area = cv2.contourArea(c)
            radius = int(np.sqrt(area / np.pi))
            if 1 < radius < iris_diameter / 2:
                normalized_pupil_size = (2 * radius) / iris_diameter
                # Convert eye_color to uint8 if it's not already
                eye_color_uint8 = eye_color.astype(np.uint8) if eye_color.dtype != np.uint8 else eye_color
                # Draw circle on the uint8 image
                cv2.circle(eye_color_uint8, center, radius, (0, 255, 0), 1)
                return eye_color_uint8, normalized_pupil_size
    return eye_color, None

In [43]:
def calculate_iris_size(landmarks, image_shape, iris_indices):
    image_height, image_width = image_shape[:2]
    iris_landmarks = [landmarks.landmark[i] for i in iris_indices]
    iris_points = [(p.x * image_width, p.y * image_height) for p in iris_landmarks]
    left_point = np.array(iris_points[0])
    right_point = np.array(iris_points[2])
    iris_diameter = np.linalg.norm(right_point - left_point)
    return iris_diameter

In [44]:
def get_eye_region(image, face_landmarks, eye_indices):
    image_height, image_width = image.shape[:2]
    # Get the coordinates of the eye landmarks
    eye_landmarks = [face_landmarks.landmark[i] for i in eye_indices]
    eye_points = np.array([
        (int(p.x * image_width), int(p.y * image_height)) for p in eye_landmarks
    ], dtype=np.int32)

    # Calculate the bounding rectangle of the eye
    x, y, w, h = cv2.boundingRect(eye_points)

    # Crop the eye region from the original image
    eye_region = image[y:y+h, x:x+w]

    # Adjust the eye landmarks to the cropped region
    eye_points_cropped = eye_points - [x, y]
    print("Original Eye Landmarks:", eye_points)
    print("Adjusted Eye Landmarks:", eye_points_cropped)

    return eye_region, eye_points_cropped

In [45]:
def save_figure(fig, filename):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    
    # Convert to RGB if the image is in RGBA mode
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    
    # Ensure the filename ends with .jpg
    if not filename.lower().endswith('.jpg'):
        filename = filename.rsplit('.', 1)[0] + '.jpg'
    
    img.save(filename, 'JPEG')
    buf.close()

In [46]:
def process_image(image_path, output_directory):
    mp_face_mesh = mp.solutions.face_mesh
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = frame.shape

    with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        EAR_THRESHOLD = 0.25
        results = face_mesh.process(frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                left_eye_indices = [33, 160, 158, 133, 153, 144]
                left_eye_points = [(face_landmarks.landmark[i].x * image_width, face_landmarks.landmark[i].y * image_height) for i in left_eye_indices]
                ear = calculate_EAR(left_eye_points)

                if ear > EAR_THRESHOLD:
                    left_iris_indices = [469, 470, 471, 472, 468]
                    iris_diameter = calculate_iris_size(face_landmarks, frame.shape, left_iris_indices)
                    left_eye_region, eye_points_cropped = get_eye_region(frame, face_landmarks, left_eye_indices)

                    if left_eye_region.size != 0:
                        eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_RGB2GRAY)
                        eye_color = left_eye_region.copy()
                        pupil_frame, normalized_pupil_size = detect_pupil(eye_gray, eye_color, iris_diameter)

                        # Convert pupil_frame to uint8 if it's not already
                        pupil_frame_uint8 = pupil_frame.astype(np.uint8) if pupil_frame.dtype != np.uint8 else pupil_frame

                        for point in eye_points_cropped:
                            cv2.circle(pupil_frame_uint8, tuple(point), 1, (0, 0, 255), -1)

                        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                        ax1.imshow(frame)
                        ax1.set_title('Original Image')
                        ax1.axis('off')
                        ax2.imshow(left_eye_region)
                        ax2.set_title('Left Eye Region')
                        ax2.axis('off')
                        ax3.imshow(pupil_frame_uint8)
                        ax3.set_title('Detected Pupil')
                        ax3.axis('off')
                        plt.tight_layout()

                        output_filename = os.path.join(output_directory, f"pupil_detection_{os.path.basename(image_path).rsplit('.', 1)[0]}.jpg")
                        save_figure(fig, output_filename)
                        plt.close(fig)

                        if normalized_pupil_size:
                            print(f"Normalized Pupil Size: {normalized_pupil_size:.2f}")
                        else:
                            print("Pupil size could not be determined.")
                        return
                else:
                    print("Left eye is closed, skipping pupil detection.")
                    return
        print("No face detected in the image.")

In [47]:
def process_directory(input_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)
    for filename in os.listdir(input_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_directory, filename)
            print(f"Processing image: {filename}")
            process_image(image_path, output_directory)
            print(f"Finished processing: {filename}\n")
    print("All images processed.")

In [48]:
# Main execution
if __name__ == "__main__":
    input_directory = r'images\inputs\artur\36points'
    output_directory = r'images\outputs\pupil_images'
    process_directory(input_directory, output_directory)

Processing image: 36points_x25_y25.jpg
Original Eye Landmarks: [[144 304]
 [155 299]
 [174 298]
 [195 308]
 [176 312]
 [157 312]]
Adjusted Eye Landmarks: [[ 0  6]
 [11  1]
 [30  0]
 [51 10]
 [32 14]
 [13 14]]
Normalized Pupil Size: 0.75
Finished processing: 36points_x25_y25.jpg

Processing image: 36points_x25_y42.jpg
Original Eye Landmarks: [[165 315]
 [176 309]
 [195 308]
 [214 318]
 [196 322]
 [178 321]]
Adjusted Eye Landmarks: [[ 0  7]
 [11  1]
 [30  0]
 [49 10]
 [31 14]
 [13 13]]
Normalized Pupil Size: 0.76
Finished processing: 36points_x25_y42.jpg

Processing image: 36points_x25_y58.jpg
Original Eye Landmarks: [[162 322]
 [174 318]
 [192 317]
 [212 326]
 [194 330]
 [175 330]]
Adjusted Eye Landmarks: [[ 0  5]
 [12  1]
 [30  0]
 [50  9]
 [32 13]
 [13 13]]
Normalized Pupil Size: 0.68
Finished processing: 36points_x25_y58.jpg

Processing image: 36points_x25_y75.jpg
Left eye is closed, skipping pupil detection.
Finished processing: 36points_x25_y75.jpg

Processing image: 36points_x25_y

In [49]:
os.makedirs(output_directory, exist_ok=True)
for filename in os.listdir(input_directory):

    image_path = os.path.join(input_directory, filename)
    print(f"Processing image: {filename}")

Processing image: 36points_x25_y25.jpg
Processing image: 36points_x25_y42.jpg
Processing image: 36points_x25_y58.jpg
Processing image: 36points_x25_y75.jpg
Processing image: 36points_x25_y8.jpg
Processing image: 36points_x25_y92.jpg
Processing image: 36points_x42_y25.jpg
Processing image: 36points_x42_y42.jpg
Processing image: 36points_x42_y58.jpg
Processing image: 36points_x42_y75.jpg
Processing image: 36points_x42_y8.jpg
Processing image: 36points_x42_y92.jpg
Processing image: 36points_x58_y25.jpg
Processing image: 36points_x58_y42.jpg
Processing image: 36points_x58_y58.jpg
Processing image: 36points_x58_y75.jpg
Processing image: 36points_x58_y8.jpg
Processing image: 36points_x58_y92.jpg
Processing image: 36points_x75_y25.jpg
Processing image: 36points_x75_y42.jpg
Processing image: 36points_x75_y58.jpg
Processing image: 36points_x75_y75.jpg
Processing image: 36points_x75_y8.jpg
Processing image: 36points_x75_y92.jpg
Processing image: 36points_x8_y25.jpg
Processing image: 36points_x8_